In [1]:
select * from menu limit 3

(3 row(s) affected)

Total execution time: 00:00:00.064

menuid,descr,descrth,sizeid,categoryid,isrecommended,isdiscontinued,price,cost,margin,username,firstintrodt,discontinueddt
201L6,Tom Yum Kung (Large),ต้มยำกุ้ง (ใหญ่),L,DI,False,False,200,80,120.0,dbo,2002-01-01T00:00:00,NULL
201M7,Tom Yum Kung (Medium),ต้มยำกุ้ง (กลาง),M,DI,True,False,100,40,60.0,dbo,2002-01-01T00:00:00,NULL
201S3,Tom Yum Kung (Small),ต้มยำกุ้ง (เล็ก),S,DI,False,False,20,8,12.0,dbo,2002-01-01T00:00:00,NULL


In [2]:
select sum(netpay) from orderhdr where date_part('year', intime)=2003

(1 row(s) affected)

Total execution time: 00:00:00.101

sum
1246449.75


In [3]:
select sum(netpay) 
    from orderhdr 
    where date_part('year', intime)=2003 and date_part('dow', intime) in (0, 6)

(1 row(s) affected)

Total execution time: 00:00:00.039

sum
524444.00


# subquery

In [30]:
SELECT
    to_char((select sum(netpay) 
    from orderhdr 
    where date_part('year', intime)=2003 and date_part('dow', intime) in (0, 6)),'9G999G999.99') "WeekendRevenue",
    to_char((select sum(netpay) from orderhdr where date_part('year', intime)=2003), '9G999G999.99') "Revenue",
    to_char((select sum(netpay) 
    from orderhdr 
    where date_part('year', intime)=2003 and date_part('dow', intime) in (0, 6)) / (select sum(netpay) from orderhdr where date_part('year', intime)=2003) * 100, '999D99"%"') "% Weekend Revenue"

(1 row(s) affected)

Total execution time: 00:00:00.051

WeekendRevenue,Revenue,% Weekend Revenue
"524,444.00","1,246,449.75",42.08%


In [32]:
select 524444, 
    to_char(5244440.75, '999G999G999D99'), 
    to_char(5244440.75 / 10, '999G999G999D99'), 
    to_char(5244440.75, '999G999G999D99') -- Group separator, D decimal locale/culture

(1 row(s) affected)

Total execution time: 00:00:00.038

Column1,to_char,to_char,to_char
524444,"5,244,440.75","524,444.08","5,244,440.75"


# Q9

In [36]:
select 
    date_part('month', intime)::int "month",
    sum(netpay) "NetPay (Eat-in)" 
    from orderhdr 
    where date_part('year', intime)=2003 and dinetypeid='EI'
    group by month

(12 row(s) affected)

Total execution time: 00:00:00.036

month,NetPay (Eat-in)
1,102803.75
2,91550.50
3,102663.00
4,88770.75
5,104751.00
6,106215.25
7,85000.75
8,96009.75
9,97742.50
10,95207.75


In [37]:
select 
    date_part('month', intime)::int "month",
    sum(netpay) "NetPay (To-Go)" 
    from orderhdr 
    where date_part('year', intime)=2003 and dinetypeid='TG'
    group by month

(12 row(s) affected)

Total execution time: 00:00:00.033

month,NetPay (To-Go)
1,5983.00
2,3448.00
3,5697.50
4,6915.75
5,6037.50
6,11322.50
7,5069.50
8,5530.00
9,6283.00
10,7743.00


In [39]:
create view vw_ei AS
    select 
        date_part('month', intime)::int "month",
        sum(netpay) "NetPay (To-Go)" 
        from orderhdr 
        where date_part('year', intime)=2003 and dinetypeid='EI'
        group by month

Commands completed successfully

Total execution time: 00:00:00.195

In [41]:
create view vw_tg AS
    select 
        date_part('month', intime)::int "month",
        sum(netpay) "NetPay (To-Go)" 
        from orderhdr 
        where date_part('year', intime)=2003 and dinetypeid='TG'
        group by month

Commands completed successfully

Total execution time: 00:00:00.011

In [49]:
select * from 
    (select 
        date_part('month', intime)::int "month",
        sum(netpay) "NetPay (To-Go)" 
        from orderhdr 
        where date_part('year', intime)=2003 and dinetypeid='EI'
        group by month) ei
    inner join 
    (select 
        date_part('month', intime)::int "month",
        sum(netpay) "NetPay (To-Go)" 
        from orderhdr 
        where date_part('year', intime)=2003 and dinetypeid='TG'
        group by month) tg
    using(month)
-- select * from vw_ei inner join vw_tg on vw_ei.month = vw_tg.month

(12 row(s) affected)

Total execution time: 00:00:00.054

month,NetPay (To-Go),NetPay (To-Go)
1,102803.75,5983.00
2,91550.50,3448.00
3,102663.00,5697.50
4,88770.75,6915.75
5,104751.00,6037.50
6,106215.25,11322.50
7,85000.75,5069.50
8,96009.75,5530.00
9,97742.50,6283.00
10,95207.75,7743.00


In [67]:
with tg AS (
    select 
        date_part('month', intime)::int "month",
        sum(netpay) "NetPay (To-Go)" 
        from orderhdr 
        where date_part('year', intime)=2003 and dinetypeid='TG'
        group by month
),
ei AS (
    select 
        date_part('month', intime)::int "month",
        sum(netpay) "NetPay (Eat-In)" 
        from orderhdr 
        where date_part('year', intime)=2003 and dinetypeid='EI'
        group by month
)
select * from ei inner join tg using(month)

(12 row(s) affected)

Total execution time: 00:00:00.040

month,NetPay (Eat-In),NetPay (To-Go)
1,102803.75,5983.00
2,91550.50,3448.00
3,102663.00,5697.50
4,88770.75,6915.75
5,104751.00,6037.50
6,106215.25,11322.50
7,85000.75,5069.50
8,96009.75,5530.00
9,97742.50,6283.00
10,95207.75,7743.00


In [73]:
with t AS (
    select 
        dinetypeid,
        date_part('month', intime)::int "month",
        sum(netpay) "NetPay" 
        from orderhdr 
        where date_part('year', intime)=2003
        group by month, dinetypeid
)
select ei.month, ei."NetPay" "NetPay (Eat-in)", tg."NetPay" "NetPay (To-Go)" 
    from t ei inner join t tg using(month)
    where ei.dinetypeid='EI' and tg.dinetypeid='TG'

(12 row(s) affected)

Total execution time: 00:00:00.045

month,NetPay (Eat-in),NetPay (To-Go)
1,102803.75,5983.00
2,91550.50,3448.00
3,102663.00,5697.50
4,88770.75,6915.75
5,104751.00,6037.50
6,106215.25,11322.50
7,85000.75,5069.50
8,96009.75,5530.00
9,97742.50,6283.00
10,95207.75,7743.00


In [47]:
select * from vw_ei inner join vw_tg using(month)

(12 row(s) affected)

Total execution time: 00:00:00.119

month,NetPay (To-Go),NetPay (To-Go)
1,102803.75,5983.00
2,91550.50,3448.00
3,102663.00,5697.50
4,88770.75,6915.75
5,104751.00,6037.50
6,106215.25,11322.50
7,85000.75,5069.50
8,96009.75,5530.00
9,97742.50,6283.00
10,95207.75,7743.00


NetPay
0-200           300
201-500         700 
501-1000
1001-2000
2000-


In [60]:
with t1 as (
    select netpay,
        case
            when netpay between 0 and 200     then '   0 -  200'
            when netpay between 201 and 500   then ' 201 -  500'
            when netpay between 501 and 1000  then ' 501 - 1000'
            when netpay between 1001 and 2000 then '1001 - 2000'
            when netpay > 2000                then '2000 -'
            else '-'
        end as "NetPay"
        from orderhdr
)
select "NetPay", count("NetPay") as count
    from t1
    group by "NetPay"
UNION
select 'total', count("NetPay") from t1
    order by "NetPay"

(6 row(s) affected)

Total execution time: 00:00:00.043

NetPay,count
0 - 200,924
201 - 500,2151
501 - 1000,1153
1001 - 2000,761
2000 -,11
total,5000


# CTE (Common Table Expression)

In [65]:
select "NetPay", count("NetPay") as count
    from (select netpay,
        case
            when netpay between 0 and 200     then '   0 -  200'
            when netpay between 201 and 500   then ' 201 -  500'
            when netpay between 501 and 1000  then ' 501 - 1000'
            when netpay between 1001 and 2000 then '1001 - 2000'
            when netpay > 2000                then '2000 -'
            else '-'
        end as "NetPay"
        from orderhdr) a
    group by "NetPay"
UNION
select 'total', count("NetPay") from (select netpay,
        case
            when netpay between 0 and 200     then '   0 -  200'
            when netpay between 201 and 500   then ' 201 -  500'
            when netpay between 501 and 1000  then ' 501 - 1000'
            when netpay between 1001 and 2000 then '1001 - 2000'
            when netpay > 2000                then '2000 -'
            else '-'
        end as "NetPay"
        from orderhdr) b
    order by "NetPay"

(6 row(s) affected)

Total execution time: 00:00:00.051

NetPay,count
0 - 200,924
201 - 500,2151
501 - 1000,1153
1001 - 2000,761
2000 -,11
total,5000


In [51]:
select netpay,
    case
        when netpay between 0 and 200 then '0-200'
        when netpay between 201 and 500 then '201-500'
        when netpay between 501 and 1000 then '501-1000'
        when netpay between 1001 and 2000 then '1001-2000'
        when netpay > 2000 then '> 2000'
        else '-'
    end as "NetPay"
    from orderhdr

(5000 row(s) affected)

Total execution time: 00:00:00.432

netpay,NetPay
1112.50,1001-2000
675.00,501-1000
520.00,501-1000
470.50,201-500
480.00,201-500
360.00,201-500
1116.00,1001-2000
425.00,201-500
1842.00,1001-2000
110.00,0-200


In [63]:
select to_char(current_date, 'Day')

(1 row(s) affected)

Total execution time: 00:00:00.038

to_char
Wednesday
